In [12]:
import imageio
import glob
import numpy as np
from sklearn.cluster import KMeans
from string import ascii_uppercase
from sklearn import preprocessing
from collections import Counter

In [6]:
def get_dataset():
    train_data = []
    data_for_labelling = []
    labeling_data_label = []
    for i in range(26):
        count = 0
        for im_path in glob.glob("Train_data/" + str(i) + "/*.jpg"):
            image = imageio.imread(im_path)
            image_data = image.ravel()
            lis = list(image_data)
            if count < 700 :
                train_data.append(lis)
            elif count < 900:
                data_for_labelling.append(lis)
                labeling_data_label.append(i)
            count += 1
    return np.array(train_data), np.array(data_for_labelling), labeling_data_label

In [7]:
train_data, data_for_labelling, labeling_data_label = get_dataset()

In [8]:
train_data=preprocessing.scale(train_data)
data_for_labelling=preprocessing.scale(data_for_labelling)

/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [9]:
print(train_data.shape)
print(data_for_labelling.shape)

(18200, 16384)
(5200, 16384)


In [10]:
print("Kmeans start")
kmeans = KMeans(n_clusters = 26, random_state=0,n_jobs=-1)

Kmeans start


In [11]:
kmeans.fit(train_data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=26, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [32]:
label_from_clusterno = dict()
remaining_labels = [i for i in range(26)]
remaining_clusters = [i for i in range(26)]
for i in range(26):
    predicted_clusters_for_i = kmeans.predict(data_for_labelling[200*i:200*(i+1)])
    predicted_clusters_for_i = Counter(predicted_clusters_for_i)
    #print(predicted_clusters_for_i)
    most_freq_cluster = predicted_clusters_for_i.most_common(1)[0][0]
    while (label_from_clusterno.get(most_freq_cluster) is not None):
        predicted_clusters_for_i.pop(most_freq_cluster)
        if len(predicted_clusters_for_i) == 0:
            break
        #print(predicted_clusters_for_i)
        most_freq_cluster = predicted_clusters_for_i.most_common(1)[0][0]
    #print(most_freq_cluster)
    if len(predicted_clusters_for_i) != 0:
        label_from_clusterno[most_freq_cluster] = i
        remaining_clusters.remove(most_freq_cluster)
        remaining_labels.remove(i)

for i in range(len(remaining_clusters)):
    label_from_clusterno[remaining_clusters[i]] = remaining_labels[i]
    
print(label_from_clusterno)

{0: 10, 1: 5, 2: 9, 3: 16, 4: 6, 5: 11, 6: 14, 7: 15, 8: 4, 9: 20, 10: 17, 11: 8, 12: 25, 13: 2, 14: 13, 15: 1, 16: 19, 17: 0, 18: 21, 19: 22, 20: 12, 21: 18, 22: 7, 23: 23, 24: 3, 25: 24}


In [37]:
from sklearn.metrics import accuracy_score
test_points = []
true_labels = []
predicted_labels = []
for path in glob.glob("./Test1char/*"):
    label = path.split('/')[-1][0]
    image = imageio.imread(path + "/Char0.jpg")
    image_data = image.ravel()
    lis = list(image_data)
    test_points.append(lis)
    true_labels.append(label)
    
test_points=np.array(test_points)
test_points=preprocessing.scale(test_points)
predicted_clusters = kmeans.predict(test_points)
for predicted_cluster in predicted_clusters:
    predicted_label = chr(label_from_clusterno[predicted_cluster]+65)
    predicted_labels.append(predicted_label)
    
print(accuracy_score(true_labels,predicted_labels))

0.20152091254752852
